### Current issue:
some of the songs are above 128(not supported), need to be clipped at the mido.MidiFile level, overwrite the PrettyMIDI __inti__ method but got issue. Skip songs currently. May need to directly called the modified pretty_midi.py.

In [4]:
import pretty_midi
import numpy as np
from os import listdir, mkdir, walk
from os.path import isfile, join
import six
import mido

In [5]:
class PrettyMIDI_clipped(pretty_midi.PrettyMIDI):
    def __init__(self, midi_file=None, resolution=220, initial_tempo=120.):
        super(pretty_midi.PrettyMIDI, self).__init__()
        
        MAX_TICK = 1e7
        
        if midi_file is not None:
            # Load in the MIDI data using the midi module
            if isinstance(midi_file, six.string_types):
                # If a string was given, pass it as the string filename
                midi_data = mido.MidiFile(filename=midi_file, clip=True)
            else:
                # Otherwise, try passing it in as a file pointer
                midi_data = mido.MidiFile(file=midi_file, clip=True)

            # Convert tick values in midi_data to absolute, a useful thing.
            for track in midi_data.tracks:
                tick = 0
                for event in track:
                    event.time += tick
                    tick = event.time

            # Store the resolution for later use
            self.resolution = midi_data.ticks_per_beat

            # Populate the list of tempo changes (tick scales)
            self._load_tempo_changes(midi_data)

            # Update the array which maps ticks to time
            max_tick = max([max([e.time for e in t])
                            for t in midi_data.tracks]) + 1
            # If max_tick is huge, the MIDI file is probably corrupt
            # and creating the __tick_to_time array will thrash memory
            if max_tick > MAX_TICK:
                raise ValueError(('MIDI file has a largest tick of {},'
                                  ' it is likely corrupt'.format(max_tick)))

            # Create list that maps ticks to time in seconds
            self._update_tick_to_time(max_tick)

            # Populate the list of key and time signature changes
            self._load_metadata(midi_data)

            # Check that there are tempo, key and time change events
            # only on track 0
            if any(e.type in ('set_tempo', 'key_signature', 'time_signature')
                   for track in midi_data.tracks[1:] for e in track):
                warnings.warn(
                    "Tempo, Key or Time signature change events found on "
                    "non-zero tracks.  This is not a valid type 0 or type 1 "
                    "MIDI file.  Tempo, Key or Time Signature may be wrong.",
                    RuntimeWarning)

            # Populate the list of instruments
            self._load_instruments(midi_data)

        else:
            self.resolution = resolution
            # Compute the tick scale for the provided initial tempo
            # and let the tick scale start from 0
            self._tick_scales = [(0, 60.0/(initial_tempo*self.resolution))]
            # Only need to convert one tick to time
            self.__tick_to_time = [0]
            # Empty instruments list
            self.instruments = []
            # Empty key signature changes list
            self.key_signature_changes = []
            # Empty time signatures changes list
            self.time_signature_changes = []
            # Empty lyrics list
            self.lyrics = []

In [6]:
def get_bar_length(pm):
    times, tempo_changes = pm.get_tempo_changes()
    tempo = int(tempo_changes[0])
    beat_per_bar = pm.time_signature_changes[0].numerator
    # length of bar in second = seconds per minute/ bpm beats per minute * number of beat in 1 bar
    bar_length = 60/tempo * beat_per_bar
    return bar_length

def get_partitions(pm):
    bar_length = get_bar_length(pm)    # manually get the length of bar
    song_length = int(pm.get_end_time())
    partitions = [i for i in np.arange(0, song_length, bar_length)]
    return partitions

def bars_split(song_path, new_path):

    try:
        pm = pretty_midi.PrettyMIDI(song_path)
    except (OSError, e):
        return None        
    
    partitions = get_partitions(pm)

    for partition in range(len(partitions)-1):
        start_time = partitions[partition]
        end_time = partitions[partition + 1]

        new_midi= pretty_midi.PrettyMIDI()
        for instr_num in range (len(pm.instruments)):
            instrument = (pm.instruments[instr_num])

            notes_velocity=[]
            notes_pitch=[]
            notes_start = []
            notes_end = []

        # the first note after start_time is called note_num
            for start_note_num in range (len(instrument.notes)):
                note = instrument.notes[start_note_num]
                start = note.start
                if start >= start_time:
                    break

            for end_note_num in range (len(instrument.notes)):
                note = instrument.notes[end_note_num]
                end = note.end
                if end > end_time:
                    break
        #record the notes and info in the original midi file 
            for k in range(start_note_num,end_note_num):
                note = instrument.notes[k]
                notes_pitch.append(note.pitch)
                notes_start.append(note.start)
                notes_end.append(note.end)
                notes_velocity.append(note.velocity)

            program = instrument.program
            is_drum = instrument.is_drum
            name = instrument.name
            inst = pretty_midi.Instrument(program=program, is_drum=is_drum, name=name)
            new_midi.instruments.append(inst)

        # copy into new midi
            for i in range (end_note_num - start_note_num):
                inst.notes.append(pretty_midi.Note(notes_velocity[i],notes_pitch[i], 
                                                   notes_start[i]-float(start_time), notes_end[i]-float(start_time)))
   
        new_midi.write(new_path + '\part'+str(partition)+'.mid')
    print('finished processing song ' + song_path)
    
    
def process_folder(song_path):    
    songs = [f for f in listdir(song_path) if isfile(join(song_path, f))]
    print('Total %  songs in the folder'%str(len(songs)))
    try:
        for song_name in songs:
            new_path = song_path +  '\\' +song_name.split('.')[0]
            mkdir(new_path)
            bars_split(song_path +'\\' +song_name, new_path)
    except:
        print ("Already exists")    
    print('Folder ' + song_path +  ' done')

In [15]:
# subdirs = [x[0] for x in walk('D:\Music\dataset\clean_midi_LAKH')][1:]
# for subdir in subdirs:
#     process_folder(subdir)

Total 2ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\.38 Special\Caught Up In You.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\.38 Special\Fantasy Girl.mid
Folder D:\Music\dataset\clean_midi_LAKH\.38 Special done
Total 1ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\10,000 Maniacs\A Campfire Song.mid
Folder D:\Music\dataset\clean_midi_LAKH\10,000 Maniacs done
Total 1ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\101 Strings\Theme From The Godfather.mid
Folder D:\Music\dataset\clean_midi_LAKH\101 Strings done
Total 10ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\10cc\Dreadlock Holiday.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\10cc done
Total 2ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\1910 Fruitgum Company\Simon Says.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\1910 Fruitgum Company

finished processing song D:\Music\dataset\clean_midi_LAKH\AL Bano\Felicita.mid
Folder D:\Music\dataset\clean_midi_LAKH\AL Bano done
Total 1ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Al Green\Let's Stay Together.mid
Folder D:\Music\dataset\clean_midi_LAKH\Al Green done
Total 2ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Al Hirt\Frankie and Johnny.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Al Hirt\When the Saints Go Marching In.mid
Folder D:\Music\dataset\clean_midi_LAKH\Al Hirt done
Total 7ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Al Jarreau\Moonlighting (Theme).1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Al Jarreau done
Total 1ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Al Jolson\You Made Me Love You.mid
Folder D:\Music\dataset\clean_midi_LAKH\Al Jolson done
Total 5ongs in the folder
finished processing song D:\Music\d

finished processing song D:\Music\dataset\clean_midi_LAKH\Alice Cooper\Be My Lover.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Alice Cooper done
Total 24ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Alice in Chains\Angry Chair.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Alice in Chains\Brother.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Alice in Chains\Dam That River.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Alice in Chains\Don't Follow.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Alice in Chains\Heaven Beside You.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Alice in Chains\I Stay Away.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Alice in Chains\Killer Is Me.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Alice in Chains\Man in a Box (live).mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Alice in Chains\No

finished processing song D:\Music\dataset\clean_midi_LAKH\Andrea Bocelli\'O mare e tu.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Andrea Bocelli\Ave Maria.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Andrea Bocelli\Caruso.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Andrea Bocelli\Con Te Partiro.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Andrea Bocelli done
Total 3ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Andrew Gold\How Can This Be Love.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Andrew Gold\Lonely Boy.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Andrew Gold done
Total 4ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Andrew Lloyd Webber\Jesus Christ Superstar Overture.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Andrew Lloyd Webber\Phantom of the Opera.mid
finished processing song D:\Music\dataset\clean

finished processing song D:\Music\dataset\clean_midi_LAKH\Assia\Elle Est a Toi.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Assia done
Total 3ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Astrud Gilberto\Once I Loved.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Astrud Gilberto\The Girl From Ipanema (Garota de Ipanema).1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Astrud Gilberto done
Total 2ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Aswad\Don't Turn Around.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Aswad\Shine.mid
Folder D:\Music\dataset\clean_midi_LAKH\Aswad done
Total 8ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Atkins Chet\Detroit City.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Atkins Chet\Georgia on My Mind.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Atkins Chet\On the Road Again.mid

...

finished processing song D:\Music\dataset\clean_midi_LAKH\Tom Jones\Delilah.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Tom Jones done
Total 1ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Tom McDermott\Heavy Henry.mid
Folder D:\Music\dataset\clean_midi_LAKH\Tom McDermott done
Total 1ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Tom Petty\You Wreck Me.mid
Folder D:\Music\dataset\clean_midi_LAKH\Tom Petty done
Total 20ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Tom Petty and The Heartbreakers\A Face in the Crowd.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Tom Petty and The Heartbreakers done
Total 1ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Tom Turpin\St. Louis Rag.mid
Folder D:\Music\dataset\clean_midi_LAKH\Tom Turpin done
Total 1ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Tommy Edwards\It's Al

finished processing song D:\Music\dataset\clean_midi_LAKH\Trio Hellenique\Zorba's Dance.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Trio Hellenique done
Total 1ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Trio Rio\New York - Rio - Tokio.mid
Folder D:\Music\dataset\clean_midi_LAKH\Trio Rio done
Total 2ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Trisha Yearwood\She's in Love With the Boy.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Trisha Yearwood\There Goes My Baby.mid
Folder D:\Music\dataset\clean_midi_LAKH\Trisha Yearwood done
Total 3ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Trovato\Amori amori.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Trovato done
Total 5ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Tupac\California Love (remix).mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Tupac\Changes.m

finished processing song D:\Music\dataset\clean_midi_LAKH\Vengaboys\Boom Boom Boom.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Vengaboys\Kiss.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Vengaboys\Uncle John From Jamaica.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Vengaboys\Up & Down.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Vengaboys\We Like to Party.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Vengaboys\We're Going to Ibiza.mid
Folder D:\Music\dataset\clean_midi_LAKH\Vengaboys done
Total 2ongs in the folder
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Ventures done
Total 1ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Vera\At This Moment.mid
Folder D:\Music\dataset\clean_midi_LAKH\Vera done
Total 1ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Vera Lynn\Land of Hope and Glory.mid
Folder D:\Music\dataset\clean_midi_LAKH\Vera Ly

finished processing song D:\Music\dataset\clean_midi_LAKH\Westlife\Swear It Again.mid
Folder D:\Music\dataset\clean_midi_LAKH\Westlife done
Total 7ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Wet Wet Wet\Angel Eyes.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Wet Wet Wet\Good Night Girl.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Wet Wet Wet done
Total 1ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Wet Willie\Keep on Smiling.mid
Folder D:\Music\dataset\clean_midi_LAKH\Wet Willie done
Total 8ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Wham!\Careless Whisper.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Wham! done
Total 9ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Whigfield\Another Day.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Whigfield done
Total 4ongs in the folder
finished processing song D:

finished processing song D:\Music\dataset\clean_midi_LAKH\Yanni\Almost a Whisper.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Yanni done
Total 2ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Yazoo\Don't Go.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Yazoo\Situation.mid
Folder D:\Music\dataset\clean_midi_LAKH\Yazoo done
Total 2ongs in the folder
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Yazz & plastic population done
Total 46ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Yes\Almost Like Love.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Yes done
Total 4ongs in the folder
finished processing song D:\Music\dataset\clean_midi_LAKH\Young Paul\Don't Dream It's Over.mid
finished processing song D:\Music\dataset\clean_midi_LAKH\Young Paul\Every Time You Go Away.1.mid
Already exists
Folder D:\Music\dataset\clean_midi_LAKH\Young Paul done
Total 1ongs in the folder
Already exist